In [1]:
# importing modules
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime
import time
import re

In [2]:
# Set up the web driver (provide the path to your Firefox web driver)
chrome_options = webdriver.FirefoxOptions()

chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--verbose")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-software-rasterizer")

driver = webdriver.Firefox(options=chrome_options)

driver.get("https://punchng.com/")

In [3]:
# Clicking the search icon
search_button = driver.find_element(By.XPATH, "(//button[@class='search-btn'])[2]")
search_button.click()

# Get the input elements
input_search = driver.find_element(By.XPATH, "//input[@name='s']")
search_button_2 = driver.find_element(By.XPATH, "//button[@class='']")

# Send the input to the webpage
input_search.send_keys('nigeria')
time.sleep(1)
search_button_2.click()

time.sleep(2)

In [4]:
# Define the number of pages you want to scrape (update this according to your needs)
num_pages_to_scrape = 2

article_urls = []

# Loop through the pages
for page in range(1, num_pages_to_scrape + 1):
    
    # Scrolling down to load more search results
    for _ in range(5):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
    
    # Find and click on each article link
    article_links = driver.find_elements(By.CLASS_NAME, "post-title")

    # Extract the URLs of the article pages
    for link in article_links:
        a_link = link.find_element(By.TAG_NAME, "a").get_attribute("href")
        article_urls.append(a_link)
        
    # Find and click the pagination button for the next page (for the "next page pagination")
    pagination_buttons = driver.find_elements(By.XPATH, "//ul[@class='pagination']/li/a[@class='page-link text-dark']")
    
    next_button_found = False  # Initialize a flag to check if the "next" button is found

    for button in pagination_buttons:
        if button.text == '»':  # Check for the "next" button
            button.click()
            next_button_found = True
            break

    # If the "next" button is not found, break out of the loop
    if not next_button_found:
        break

In [6]:
len(article_urls)

10

In [7]:
# Define today's date and yesterday's date
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Initialize an empty DataFrame
df = pd.DataFrame(columns=["headline", "author", "post_date", "Content"])

# Loop through the article URLs
for url in article_urls:
    driver.get(url)

    # Wait for the article page to load
    wait = WebDriverWait(driver, 10)

    date_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "post-date")))
    date_text = date_element.text

    # Extract the date and compare it with today and yesterday
    date_match = re.search(r'(\d{1,2}(?:st|nd|rd|th)\s\w+\s\d{4})', date_text)

    if date_match:
        date_str = date_match.group(0)
        date_str = re.sub(r'st|nd|rd|th', '', date_str)
        article_date = datetime.datetime.strptime(date_str, "%d %B %Y").date()

        # Check if the article date is within the desired date range
        if article_date >= yesterday and article_date <= today:
            # scrapping headlines
            headline = driver.find_element(By.CLASS_NAME, 'post-title').text
            # scrapping author of the article
            if driver.find_element(By.CLASS_NAME, 'post-author'):
                author = driver.find_element(By.CLASS_NAME, 'post-author').text
            else:
                author = 'None'

            # scrapping published date
            post_date = driver.find_element(By.CLASS_NAME, 'post-date').text

            # scrapping content of the article
            raw_content = driver.find_elements(By.CLASS_NAME, 'post-content')
            content = ''
            for con in raw_content:
                content += con.text
            
            # storing all data in to dataframe
            df = df.append({"headline": headline, "author": author, "post_date": post_date, "Content":content}, ignore_index=True)

            # print('one article is done')
        


C:\Users\Admin\AppData\Local\Temp\ipykernel_5160\893942554.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"headline": headline, "author": author, "post_date": post_date, "Content":content}, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_5160\893942554.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"headline": headline, "author": author, "post_date": post_date, "Content":content}, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_5160\893942554.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"headline": headline, "author": author, "post_date": post_date, "Content":content}, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipyker

In [8]:
len(df)

10

In [10]:
df.sample(2)

headline  \
1  Nigeria should stand for the truth in Israel-Hamas war – MUSWEN Executive Secretary   
9           Nigeria’s future bleak without restructuring – Afenifere, Ohanaeze, others   

                 author          post_date  \
1  By \nFarida Mohammed  29th October 2023   
9   By \nGodfrey George  28th October 2023   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [11]:
# storing data in to excel file
df.to_excel('Punchng_articles.xlsx', index=False)

In [12]:
# closing web driver
driver.quit()